In [1]:
import sys
import ROOT
import numpy as np
import pandas as pd
import root_pandas
import seaborn as sb
import matplotlib.pyplot as plt
import uproot
import time

from itertools import product

#from root_numpy import root2array

from keras.models import Sequential
from keras.layers import Dense

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, auc

from bayes_opt import BayesianOptimization
from sklearn.preprocessing import QuantileTransformer
import pickle

Welcome to JupyROOT 6.16/00


/Users/jonamotta/anaconda3/envs/python_root/lib/python2.7/site-packages/root_numpy/__init__.py:46: RuntimeWarning: numpy 1.16.3 is currently installed but you installed root_numpy against numpy 1.9.3. Please consider reinstalling root_numpy for this numpy version.
  RuntimeWarning)
Using TensorFlow backend.


In [2]:
#########################################
### PREPARE DFs FOR TRAINING AND TEST ###
#########################################


# some of these features are taken from the LHCb paper and have to be computed and added to the dataframes
features = [
    'm2_miss', 
    'muE_Brf', 
    'q2', 
    'pT_miss',
    'mu_pt'     ,
    'mu_eta'    ,
    'mu_phi'    ,
    #'mu_charge' ,
    'mu1_pt'    ,
    'mu1_eta'   ,
    'mu1_phi'   ,
    #'mu1_charge',
    'mu2_pt'    ,
    'mu2_eta'   ,
    'mu2_phi'   ,
    #'mu2_charge',
]

mu_events = pd.read_csv('mu_events.csv')
tau_events = pd.read_csv('tau_events.csv')

#add the column target to both dataframes
mu_events['target'] = 0
tau_events['target'] = 1

# concatenate the two samples
dataset = pd.concat([mu_events, tau_events],sort=False)

# shuffle and split train/test
train, test = train_test_split(dataset, test_size=0.85, random_state=1986, shuffle=True)

# X and Y on the training sample
X = pd.DataFrame(train, columns=features)
Y = pd.DataFrame(train, columns=['target'])

In [3]:
##################
### PREPROCESS ###
##################


qt = QuantileTransformer(output_distribution='normal', random_state=1986)
qt.fit(X[features])
transformedX = qt.transform(X[features])
pickle.dump( qt, open( 'quantile_tranformation.pck', 'w' ) )

In [4]:
####################################################
### FUNCTION FOR BAYESIAN OPTIMIZATION OF THE NN ###
####################################################


def BO_function(features,transformedX,Y,test,pbounds,init_points,n_iter):
    start = time.time()
    def NN_function(n_layers,units_perlayer,batch_size):
    
        #optimizer_fcts = ['nadam', 'adamax', 'adam', 'adadelta', 'adagrad', 'rmsprop', 'sgd']
        #activation_fcts = ['softmax', 'elu', 'selu', 'relu', 'softplus', 'softsign', 'tanh', 'sigmoid', 
        #                   'hard_sigmoid', 'exponential']
        #metrics_fcts = ['binary_accuracy', 'categorical_accuracy', 'sparse_categorical_accuracy', 
        #                'top_k_categorical_accuracy', 'sparse_top_k_categorical_accuracy']
        #loss_fcts = ['mean_squared_error', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error',
        #            'squared_hinge', 'hinge', 'categorical_hinge', 'logcosh', 'categorical_crossentropy',
        #            'sparse_categorical_crossentropy', 'binary_crossentropy', 'kullback_leibler_divergence',
        #            'poisson', 'cosine_proximity']
        
        # I want units_perlayer to be a multiple of 2 -> I always take le lower multiple of 2 starting from the
        # float that the algorith is giving me
        if units_perlayer%2 < 1:
            units_perlayer = int(units_perlayer)
        else:
            units_perlayer = int(units_perlayer-1)

        # define the model
        model = Sequential()
        for i in range(int(n_layers)):
            model.add(Dense(units_perlayer, input_dim=len(features),activation='relu'))
        model.add(Dense(1,activation='sigmoid'))

        # compile the model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        # fit the model
        model.fit(transformedX, Y, epochs=20, batch_size=int(batch_size), validation_split=0.6,verbose=0)

        # evaluate the model
        scores = model.evaluate(transformedX, Y,verbose=0)

        # calculate predictions on the test sample
        x = pd.DataFrame(test, columns=features)
        qt = pickle.load(open( 'quantile_tranformation.pck', 'r' ))
        transformedx = qt.transform(x[features])
        y = model.predict(transformedx)
    
        # create this random in order to avoid repetitions in the insertion of the score of the NN (nnS)
        k = np.random.normal(100,20)
        
        # add the score to the test sample dataframe
        test.insert(len(test.columns), 'nnS'+str(k), y)

        # let sklearn do the heavy lifting and compute the ROC curves for you
        fpr, tpr, wps = roc_curve(test.target, test['nnS'+str(k)])

        # compute the auc
        auroc = auc(fpr, tpr)

        # compute Gini index
        gini_index = (auroc-0.5)*2
        
        return np.log((auroc*scores[1])**4)

    optimizer = BayesianOptimization(
        f = NN_function,
        pbounds = pbounds,
    )
    
    # optimize
    optimizer.maximize(
        init_points=init_points,
        n_iter=n_iter,
        alpha = 1e-3
    )
    
    print optimizer.max
    
    end = time.time()
    print 'Running time of the Bayesian Optimization = %.1f'%(end - start)

In [5]:
#########################################
### OPTIMIZE THE PARAMETERS OF THE NN ###
#########################################


# bounded region of parameter space
pbounds = {'n_layers': (3, 10), 'units_perlayer': (16, 128), 'batch_size': (5, 5000)}
# , 'optimizer': (0,7), 'activation': (0,10), 'loss': (0,13)




# call the function that creates the optimizer
BO_function(features,transformedX,Y,test,pbounds,40,60)

|   iter    |  target   | batch_... | n_layers  | units_... |
-------------------------------------------------------------
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
|  1        | -0.7062   |  3.105e+0 |  3.458    |  81.93    |
|  2        | -0.7885   |  3.33e+03 |  4.436    |  26.89    |
|  3        | -0.7114   |  4.936e+0 |  7.535    |  66.28    |
|  4        | -0.7125   |  4.007e+0 |  4.95     |  70.61    |
|  5        | -0.705    |  1.97e+03 |  4.972    |  61.5     |
|  6        | -0.7046   |  1.025e+0 |  8.725    |  40.12    |
|  7        | -1.078    |  3.963e+0 |  9.113    |  18.07    |
|  8        | -0.977    |  3.579e+0 |  1.066    |  41.55    |
|  9        | -0.7132   |  3.397e+0 |  4.365    |  61.23    |
|  10       | -0.7049   |  2.477e+0 |  7.115    |  127.1    |
|  11       | -0.7015   |  1.704e+0 |  7.398    |  58.99    |
|  12       | -1.05     |  3.217e+0 |  1.366    |  43.86    |
|  13       

|  74       | -0.6967   |  2.366e+0 |  10.0     |  128.0    |
|  75       | -0.6986   |  3.465e+0 |  10.0     |  128.0    |
|  76       | -0.6938   |  5.0      |  1.0      |  16.0     |
|  77       | -0.6996   |  493.4    |  1.0      |  128.0    |
|  78       | -0.7597   |  2.576e+0 |  1.0      |  128.0    |
|  79       | -0.6944   |  2.857e+0 |  10.0     |  128.0    |
|  80       | -0.7822   |  819.1    |  1.0      |  16.0     |
|  81       | -1.065    |  3.042e+0 |  10.0     |  16.0     |
|  82       | -0.8552   |  1.139e+0 |  1.0      |  16.0     |
|  83       | -0.7012   |  3.371e+0 |  10.0     |  128.0    |
|  84       | -0.7078   |  168.4    |  1.0      |  16.0     |
|  85       | -0.6948   |  268.4    |  1.0      |  128.0    |
|  86       | -0.7335   |  895.0    |  10.0     |  128.0    |
|  87       | -0.716    |  1.376e+0 |  1.0      |  128.0    |
|  88       | -0.6932   |  3.586e+0 |  10.0     |  128.0    |
|  89       | -0.7404   |  553.4    |  1.0      |  16.0     |
|  90   

2019-07-18 14:34:54.230301: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
